In [1]:
from aiida.orm import Code, StructureData, load_node
from aiida.plugins import DataFactory
from aiida import orm
from aiida.plugins import CalculationFactory
from aiida.engine import launch
from aiida.orm import load_group
import ase.io # 構造を読み込む用
from aiida import load_profile
load_profile()

Profile<uuid='ce0a34b5764048fa9cb47539416c01b7' name='ta'>

In [16]:
codename = 'qe-6.6-ph@ohtaka'
code = Code.get_from_string(codename)

# codeから，対応するbuilderを取得
builder = code.get_builder()


In [21]:
filename="/Users/amano/works/research/jsr_utokyo/cpmd/02_GaAs/GaAs_mp-2534_conventional_standard.cif"
#filename="02_GaAs.cif"


# load structure (currently only cif is available)
ase_structure=ase.io.read(filename, format="cif")
import numpy as np
P=np.diag(np.array([2,2,2]))
import ase
test=ase.build.make_supercell(ase_structure, P)

In [20]:
print(ase_structure.get_cell())
print(test.get_cell())

Cell([5.750182, 5.750182, 5.750182])
Cell([11.500364, 11.500364, 11.500364])


In [3]:
# 計算条件は構造だけを最適化構造から取得
first_calculation = orm.load_node(707)
structure=first_calculation.outputs.output_structure


In [4]:
print(structure.attributes)


{'cell': [[5.5961558311337, 0.0, 0.0], [0.0, 5.5961558311337, 0.0], [0.0, 0.0, 5.5961558311337]], 'pbc1': True, 'pbc2': True, 'pbc3': True, 'kinds': [{'mass': 69.723, 'name': 'Ga', 'symbols': ['Ga'], 'weights': [1.0]}, {'mass': 74.921595, 'name': 'As', 'symbols': ['As'], 'weights': [1.0]}], 'sites': [{'position': [0.0, 0.0, 0.0], 'kind_name': 'Ga'}, {'position': [0.0, 2.7980779155169, 2.7980779155169], 'kind_name': 'Ga'}, {'position': [2.7980779155169, 0.0, 2.7980779155169], 'kind_name': 'Ga'}, {'position': [2.7980779155169, 2.7980779155169, 0.0], 'kind_name': 'Ga'}, {'position': [1.3990390324968, 1.3990390324968, 1.3990390324968], 'kind_name': 'As'}, {'position': [1.3990390324968, 4.197116798537, 4.197116798537], 'kind_name': 'As'}, {'position': [4.197116798537, 1.3990390324968, 4.197116798537], 'kind_name': 'As'}, {'position': [4.197116798537, 4.197116798537, 1.3990390324968], 'kind_name': 'As'}]}


In [17]:
import aiida.tools.data.array.kpoints.main
test2=aiida.tools.data.array.kpoints.main.get_explicit_kpoints_path(structure,method='seekpath')

In [18]:
test2["explicit_kpoints"]

<KpointsData: uuid: ab995fca-526f-4707-a18d-6bb7fa67b001 (unstored)>

In [23]:
print(test2["parameters"]["point_coords"])

print(test2["parameters"]["path"])

{'GAMMA': [0.0, 0.0, 0.0], 'X': [0.5, 0.0, 0.5], 'L': [0.5, 0.5, 0.5], 'W': [0.5, 0.25, 0.75], 'W_2': [0.75, 0.25, 0.5], 'K': [0.375, 0.375, 0.75], 'U': [0.625, 0.25, 0.625]}
[('GAMMA', 'X'), ('X', 'U'), ('K', 'GAMMA'), ('GAMMA', 'L'), ('L', 'W'), ('W', 'X')]


In [7]:

test3=aiida.tools.data.array.kpoints.seekpath.get_kpoints_path(structure,{})["parameters"]

In [9]:
test3.dict.path

[('GAMMA', 'X'),
 ('X', 'U'),
 ('K', 'GAMMA'),
 ('GAMMA', 'L'),
 ('L', 'W'),
 ('W', 'X')]

In [35]:
test3.dict.reciprocal_primitive_lattice

[[-1.122768110248764, 1.122768110248764, 1.122768110248764],
 [1.122768110248764, -1.122768110248764, 1.122768110248764],
 [1.122768110248764, 1.122768110248764, -1.122768110248764]]

In [10]:
test3.dict.point_coords

{'GAMMA': [0.0, 0.0, 0.0],
 'X': [0.5, 0.0, 0.5],
 'L': [0.5, 0.5, 0.5],
 'W': [0.5, 0.25, 0.75],
 'W_2': [0.75, 0.25, 0.5],
 'K': [0.375, 0.375, 0.75],
 'U': [0.625, 0.25, 0.625]}

In [11]:
test3.dict

array([31, 33, 31, 33, 31, 33, 31, 33], dtype=int32)

In [8]:
# -*- coding: utf-8 -*-
###########################################################################
# GaAs bulk ph calculation
# https://aiida-quantumespresso.readthedocs.io/en/latest/user_guide/get_started/examples/pw_tutorial.html
#
# in ph calculation 

from aiida.orm import Code, StructureData
from aiida.plugins import DataFactory
from aiida import orm
from aiida.plugins import CalculationFactory
from aiida.engine import launch
from aiida.orm import load_group
import ase.io # 構造を読み込む用
import aiida.tools.data.array.kpoints.seekpath

# ===========   code  ===============
# codename = 'qe-7.0-pw@sauron'
codename = 'qe-6.6-matdyn@ohtaka'
code = Code.get_from_string(codename)

# codeから，対応するbuilderを取得
builder = code.get_builder()

# ======= parent q2r calc ===========
parent_ID= 1008
# ===============================

# ======= parent qw calc ===========
pw_ID= 937
# ===============================


# inputs
parameters = orm.Dict({
    "INPUT":{"asr":"simple",
             },
             })

# builder
builder.parameters=orm.Dict(dict=parameters)

# force_constants=load_node(parent_ID).outputs.force_constants
# input_test=[(force_constants.uuid, force_constants.filename, force_constants.filename)]
# builder.parent_folder=input_test


builder?


Type:            ProcessBuilder-e2c0b612-2738-45d5-80dd-e2d7dc9649fa
String form:     {'metadata': {'options': {'stash': {}}}, 'code': <Code: Remote code 'qe-6.6-matdyn' on ohtaka, pk <...> -a574-5f3c8a67e6f8>, 'parameters': <Dict: uuid: aec80e0c-6222-4e6c-bd0c-d5f2283cca73 (unstored)>}
Length:          3
File:            ~/anaconda3/envs/aiida/lib/python3.10/abc.py
Docstring:       <no docstring>
Class docstring: A process builder that helps setting up the inputs for creating a new process.
Init docstring: 
Construct a `ProcessBuilder` instance for the given `Process` class.

:param process_class: the `Process` subclass


In [9]:

#builder.parent_calculation=load_node(parent_ID).outputs.force_constants
builder.parent_folder=load_node(parent_ID).outputs.remote_folder

builder.force_constants=load_node(parent_ID).outputs.force_constants

builder?

Type:            ProcessBuilder-e2c0b612-2738-45d5-80dd-e2d7dc9649fa
String form:     {'metadata': {'options': {'stash': {}}}, 'code': <Code: Remote code 'qe-6.6-matdyn' on ohtaka, pk <...> , 'force_constants': <ForceConstantsData: uuid: ba306eb2-eed3-40ad-8a8c-d984d845f1f4 (pk: 1011)>}
Length:          5
File:            ~/anaconda3/envs/aiida/lib/python3.10/abc.py
Docstring:       <no docstring>
Class docstring: A process builder that helps setting up the inputs for creating a new process.
Init docstring: 
Construct a `ProcessBuilder` instance for the given `Process` class.

:param process_class: the `Process` subclass


In [ ]:

# 構造を取得して対応するk点を取得
struc=load_node(pw_ID).inputs.structure
# kpath=aiida.tools.data.array.kpoints.main.get_explicit_kpoints_path(struc)
# print("")
# print(kpath["parameters"].dict.path)
#print(kpath["explicit_kpoints"].labels)
#print("")
#print(kpath["parameters"])
#print("")
#print(kpath["primitive_structure"])
#print("")

kpath_explicit=aiida.tools.data.array.kpoints.seekpath.get_explicit_kpoints_path(struc,{})
print(" ----------------- ")
print(" get explicit k-path ")
print(kpath_explicit["parameters"]["point_coords"])
print("")
print(kpath_explicit["parameters"]["path"])
print("")
builder.kpoints=kpath_explicit["explicit_kpoints"]

# additional setting (necessary)
builder.metadata.options.resources = {'num_machines': 4, 'tot_num_mpiprocs':128}
builder.metadata.options.withmpi = True
builder.metadata.options.queue_name = "i8cpu"
builder.metadata.options.max_wallclock_seconds = 24*60*60

# additional setting 1 (optional)
builder.metadata.label = "q2r GaAs"
builder.metadata.description = "q2r calculation GaAs 3*3*3 q-points for bands from {}".format(load_node(parent_ID).pk)
builder.metadata.options.output_filename = "GaAs.q2r.in"
builder.metadata.options.output_filename = "GaAs.q2r.out"

builder.metadata.options.custom_scheduler_commands = '''
#SBATCH -J matdyn
#SBATCH --mail-type=all        #available type:BEGIN, END, FAIL, REQUEUE, ALL
#SBATCH --mail-user=tragic44cg@icloud.com
'''
builder.metadata.options.prepend_text = '''
# output calculation settings
echo
echo    START DATE       : `date`
echo   SLURM_JOBID       : ${SLURM_JOBID}
echo SLURM_SUBMIT_DIR    : ${SLURM_SUBMIT_DIR}
echo SLURM_CPUS_PER_TASK : ${SLURM_CPUS_PER_TASK}
echo SLURM_JOB_NUM_NODES : ${SLURM_JOB_NUM_NODES} # num of nodes allocated to the job
echo   SLURM_NTASKS      : ${SLURM_NTASKS}
'''

builder.metadata.dry_run = True
builder.metadata.store_provenance = False

# job submission to daemon
job=launch.submit(builder)

# after submission
print('launched WorkChain<{}> for structure {}'.format(job.pk, load_node(pw_ID).inputs.structure.get_formula()))
print("Use `verdi process list` or `verdi process show {}` to check the progress".format(job.pk))



